In [9]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC 
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from datetime import datetime
import mysql.connector

# MySQL connection details
host = 'localhost'
user = 'root'
password = '1234'
database = 'redbus'

# Connect to MySQL
conn = mysql.connector.connect(
    host=host,
    user=user,
    password=password,
    database=database
)

cursor = conn.cursor()

drop_table_query = 'DROP TABLE IF EXISTS bus_schedule'
cursor.execute(drop_table_query)

# Create table query
create_table_query = '''
CREATE TABLE IF NOT EXISTS bus_schedule (
    id INT AUTO_INCREMENT PRIMARY KEY,
    state VARCHAR(255),
    route VARCHAR(255),
    bus_name VARCHAR(255),
    bus_type VARCHAR(255),
    departure_date VARCHAR(255),
    departure_time TIME,
    duration VARCHAR(255),
    arrival_date VARCHAR(255),
    arrival_time TIME,
    rating VARCHAR(100),
    price  FLOAT,
    seats_available INT
)'''

# Execute the query
cursor.execute(create_table_query)

driver = webdriver.Chrome()
driver.get('https://www.redbus.in/')
driver.maximize_window()
actions = ActionChains(driver)
state_elements = driver.find_elements(By.CLASS_NAME, 'rtcCards')

for index, state in enumerate(state_elements):
    state_elements = driver.find_elements(By.CLASS_NAME, 'rtcCards')
    state = state_elements[index]
    state_name = state.text.split("\n")
    state_name = state_name[0]
    print(state_name)
    actions.move_to_element(state).click().perform()
    cur_web = driver.current_url
    if cur_web == "https://www.redbus.in/":
        print("website not loaded")
    else:
        print("Loaded")
    
    wait = WebDriverWait(driver, 10)
    pagination_container = wait.until(EC.presence_of_element_located((By.CLASS_NAME, "DC_117_paginationTable")))
    active_tab = pagination_container.find_element(By.CLASS_NAME, "DC_117_pageActive")
    all_tabs = pagination_container.find_elements(By.CLASS_NAME, "DC_117_pageTabs")
    for a in range(len(all_tabs)+1):
        # print(len(all_tabs))
        actions = ActionChains(driver)
        wait = WebDriverWait(driver, 10)
        # print(f"Page {a}")
        current_url = driver.current_url
        print(f"Current URL: {current_url}")
        if a != 0 and a < len  (all_tabs)+1:
            # print (a)
            pagination_container = wait.until(EC.presence_of_element_located((By.CLASS_NAME, "DC_117_paginationTable")))
            active_tab = pagination_container.find_element(By.CLASS_NAME, "DC_117_pageActive")
            all_tabs = pagination_container.find_elements(By.CLASS_NAME, "DC_117_pageTabs")
            next_tab = all_tabs[a-1]
            actions.move_to_element(next_tab).click().perform() 
            Route_title=driver.find_elements(By.XPATH,"//a[@class='route']")
            Route_title_array = []
            
            for x in Route_title:
                # print(j.text)
                Route_title_array.append(x.text)
            for route in Route_title_array:
                print(route)
                route_name = route
                route_link = wait.until(EC.element_to_be_clickable((By.LINK_TEXT,route)))
                actions.move_to_element(route_link).click().perform()
                print ("clicked on route link")
                time.sleep(5)
                scrolling=True
                while scrolling:
                    old_page_source=driver.page_source
                    body = driver.find_element(By.TAG_NAME,'body')
                    driver.execute_script("window.scrollBy(0, 10000);")
                    time.sleep(0.05)
                    new_page_source = driver.page_source

                    if new_page_source == old_page_source:
                        scrolling = False
                        body.send_keys(Keys.HOME)
                    
                bus_data = driver.find_elements(By.XPATH,"//div[@class='clearfix bus-item-details']")
                # print(len(bus_data))
                bus_data_list=[]
                for j in bus_data:
                    bus_data_list_split= j.text.split("\n")
                    presence_of_next_day = 1 if "Next day" in bus_data_list_split else 0
                    if presence_of_next_day == 1:
                        dep_day = "Next day"
                        bus_data_list_split.remove("Next day")
                    else:
                        dep_day = "Today"
                    bus_name = bus_data_list_split[0]
                    bus_type = bus_data_list_split[1]
                    dep_time = bus_data_list_split[2]
                    dep_time = datetime.strptime(dep_time, "%H:%M").time()
                    tra_time = bus_data_list_split[4]
                    ari_time = bus_data_list_split[5]

                    ari_day  = bus_data_list_split[6]
                    date_format = "%d-%b"
                    # Try to parse the string into a date object
                    try:
                        valid_date = datetime.strptime(ari_day, date_format)
                        ari_day = ari_day
                    except ValueError:
                        ari_day = "-"
                    ari_time = datetime.strptime(ari_time, "%H:%M").time()

                    if ari_day == "-":
                        rating   = (bus_data_list_split[7])
                    else:
                        rating   = (bus_data_list_split[8])

                    Price = next((element for element in bus_data_list_split if "INR" in element), None)
                    Price = Price.split(" ")
                    Price = float(Price[len(Price)-1])
                    ava_seat = next((element for element in reversed(bus_data_list_split) if "Seat" in element), None)
                    ava_seat = ava_seat.split(' ')
                    ava_seat = int(ava_seat[0])
                    print(bus_name, bus_type,dep_day, dep_time, tra_time, ari_day, ari_time, rating, Price,ava_seat)
                    
                    insert_query = '''
                    INSERT INTO bus_schedule (
                    state, route,bus_name, bus_type,departure_date, departure_time, duration,arrival_date, arrival_time, rating, price, seats_available
                    ) VALUES (%s,%s,%s, %s,%s,%s, %s, %s, %s, %s, %s, %s)
                    '''
                    data = (state_name, route_name,bus_name, bus_type, dep_day, dep_time, tra_time,ari_day, ari_time, rating, Price,ava_seat)

                    cursor.execute(insert_query, data)

                    conn.commit() 

                time.sleep(2)
                driver.get(current_url)
                # print (a)   
                pagination_container = wait.until(EC.presence_of_element_located((By.CLASS_NAME, "DC_117_paginationTable")))
                active_tab = pagination_container.find_element(By.CLASS_NAME, "DC_117_pageActive")
                all_tabs = pagination_container.find_elements(By.CLASS_NAME, "DC_117_pageTabs")
                next_tab = all_tabs[a-1]
                actions.move_to_element(next_tab).click().perform() 
        else:
            print("over")
    driver.get('https://www.redbus.in/')
    time.sleep(2)

APSRTC
Loaded
Current URL: https://www.redbus.in/online-booking/apsrtc/?utm_source=rtchometile
over
Current URL: https://www.redbus.in/online-booking/apsrtc/?utm_source=rtchometile
Vijayawada to Hyderabad
clicked on route link
IntrCity SmartBus A/C Seater / Sleeper (2+1) Today 23:45:00 06h 15m 19-Jul 06:00:00 4.6 446.0 40
IntrCity SmartBus A/C Seater / Sleeper (2+1) Today 23:25:00 07h 00m 19-Jul 06:25:00 4.6 535.0 37
FRESHBUS Electric A/C Seater (2+2) Today 22:30:00 06h 50m 19-Jul 05:20:00 4.6 450.0 32


ValueError: time data '19-Jul' does not match format '%H:%M'